In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
import chromadb

import os
import time
import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from helpers import pinecone_connect


index = pinecone_connect()

embedding_model = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2")

samples = ['Blackpool_Borough_Council',
           'Crawley_Borough_Council',
           'Isle_of_Wight_Council',
           'Hastings_Borough_Council',
           'South_Oxfordshire_District_Council',
           'London_Borough_of_Newham']

samples = os.listdir('txts')
samples = sorted(samples)

output = pd.DataFrame(columns=('lpa', 'score', 'word_count'))

query_term = 'aging population'

for sample in samples:
    # embed query
    query = embedding_model.embed_documents(
        [query_term],
    )

    f = open('txts/' + sample)
    string = f.read()
    word_count = len(string.split())

    # Query
    results = index.query(
        vector=query,
        filter={
            "LPA": {"$eq": sample},
        },
        top_k=20,
        include_metadata=True
    )

    total_score = 0
    for result in results['matches'][0:20]:
        total_score += result['score']

    '''for result in results['matches'][0:5]:
        print(result['metadata']['text'])
        print('\n')'''

    lpa_name = sample.replace('.txt', '')

    output.loc[len(output)] = (
        {'lpa': lpa_name.replace('_', ' '), 'score': total_score, 'word_count': float(word_count)})


highest = output.sort_values('score', ascending=False).head(10)

lowest = output.sort_values(
    'score', ascending=True).query('word_count > 15000 and score >5').head(10)


combined = pd.concat([highest, lowest])

combined.to_csv(query_term + '.csv', index=False)

display(combined)

,lpa,score,word_count
113,North Lincolnshire Council,11.338803,147066.0
63,Fylde Borough Council,10.767611,101573.0
38,Crawley Borough Council,10.609713,142388.0
114,North Norfolk District Council,10.592075,119950.0
119,Northumberland County Council,10.587782,158706.0
71,Havant Borough Council,10.514498,72866.0
138,Shropshire Council,10.242911,148424.0
77,Isles of Scilly Council,10.096234,41332.0
17,Brighton and Hove City Council,10.059563,89187.0
159,Suffolk Coastal District Council,9.984921,194867.0


In [4]:
import os
samples = os.listdir('txts')
samples = sorted(samples)


for sample in samples:
    sample_no_extn = sample.replace('.txt', '')
    sample_human = sample_no_extn.replace('_', ' ')

    print(f'{{label:"{sample_human}", value:"{sample_no_extn}"}},')

{label:"Allerdale Borough Council", value:"Allerdale_Borough_Council"},
{label:"Arun District Council", value:"Arun_District_Council"},
{label:"Ashfield District Council", value:"Ashfield_District_Council"},
{label:"Barnsley Metropolitan Borough Council", value:"Barnsley_Metropolitan_Borough_Council"},
{label:"Basildon Borough Council", value:"Basildon_Borough_Council"},
{label:"Bassetlaw District Council", value:"Bassetlaw_District_Council"},
{label:"Bath and North East Somerset Council", value:"Bath_and_North_East_Somerset_Council"},
{label:"Bedford Borough Council", value:"Bedford_Borough_Council"},
{label:"Birmingham City Council", value:"Birmingham_City_Council"},
{label:"Blaby District Council", value:"Blaby_District_Council"},
{label:"Blackburn with Darwen Borough Council", value:"Blackburn_with_Darwen_Borough_Council"},
{label:"Blackpool Borough Council", value:"Blackpool_Borough_Council"},
{label:"Bolton Metropolitan Borough Council", value:"Bolton_Metropolitan_Borough_Council